# Image Segmantation for Pothole Detection
(we know this would be very helpful in Romania expecially)

Let's import the dataset:

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("farzadnekouei/pothole-image-segmentation-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Tudor\.cache\kagglehub\datasets\farzadnekouei\pothole-image-segmentation-dataset\versions\2
